This notebook was created by [William Mattingly](https://datascience.si.edu/people/dr-william-mattingly) for the 2022 Text Analysis Pedagogy Institute, with support from the [National Endowment for the Humanities](https://neh.gov), [JSTOR Labs](https://labs.jstor.org/), and [University of Arizona Libraries](https://new.library.arizona.edu/).

This notebook is adapted by Zhuo Chen under [Creative Commons CC BY License](https://creativecommons.org/licenses/by/4.0/).

For questions/comments/improvements, email zhuo.chen@ithaka.org or nathan.kelber@ithaka.org.<br />
____

# Multilingual NER 3

This is lesson 3 in the educational series on named entity recognition. 

**Audience:** Teachers / Learners / Researchers

**Use case:** Explanation

**Difficulty:** Intermediate

**Completion time:** 90 minutes

**Knowledge Required:** 

* Python basics ([start learning Python basics](./python-basics-1.ipynb))
* [Python intermediate 4](./python-intermediate-4.ipynb) (OOP, classes, instances, inheritance)
* [Regular expressions](./regular-expressions.ipynb) (re, character classes)

**Knowledge Recommended:**

* Basic file operations ([start learning file operations](./python-intermediate-2.ipynb))
* Data cleaning with `Pandas` ([start learning Pandas](./pandas-1.ipynb))


**Learning Objectives:**
After this lesson, learners will be able to:

* Understand word embeddings
* Understand Machine Learning generally
* Understanding of how to do NER ML in spaCy 3
___

# Install libraries

In [ ]:
!pip3 install spacy # for NLP
!pip3 install pandas # for making tabular data
!python3 -m spacy download en_core_web_sm # for English NER
!python3 -m spacy download en_core_web_md # for showing the word vectors

# Introduction to word embeddings

How do we represent word meanings in NLP? Well, we use numerical values. **Word embeddings** are vector representations of words.

## Distributional hypothesis

Word embeddings is inspired by the **distributional hypothesis** was proposed by Harris ([1954](https://doi.org/10.1080/00437956.1954.11659520)). This theory could be summarized as: words that have similar context will have similar meanings.

What does "context" mean in word mebeddings? Basically, "context" means the neighboring words of a target word. 

Consider the following example. If we choose "village" as the target word and choose a fixed size context window of 2, the two words before "village" and the two words after "village" will constitute the context of the target word.

Treblinka is **a small** **<span style="color: blue;">village</span>** **in Poland.**



## Word2Vec

Google’s pre-trained word2vec model includes word vectors for a vocabulary of 3 million words and phrases that they trained on roughly 100 billion words from a Google News dataset. The vector length is 300 features, which means each of the 3 million words in the vocabulary is represented by a vector with 300 floating numbers. Word2Vec is one of the most popular techniques to learn word embeddings.

The training samples are the (target, context) pairs from the text data. For example, suppose your source text is the sentence "The quick brown fox jumps over the lazy dog". If you choose "quick" as your target word and have set a context window of size 2, you will get three training samples for it, i.e. (quick, the), (quick, brown) and (quick fox).   

**McCormick, C**. (2016, April 19). Word2Vec Tutorial - The Skip-Gram Model. Retrieved from http://www.mccormickml.com

The word2vec model is trained to accomplish the following task: given the input word $w_{1}$, for each word in our vocab $w_{2}$, how likely $w_{2}$ is a context word of $w_{1}$.

The network is going to learn the statistics from the number of times each (target, context) shows up. So, for example, if you have a text about kings, queens and kingdoms, the network is probably going to get many more training samples of ("King", "Queen") than ("King", "kangaroo"). Therefore, if you give your trained model the word "King" as input, then it will output a much higher probability for "Queen" than it will for "kangaroo".

## SpaCy

We have used the small English model from spaCy. Actually, there are medium size and large size English model from spaCy as well. Both are trained using the word2vec family of algorithms.

In [ ]:
import spacy
# Load the medium size English model from spaCy
nlp = spacy.load('en_core_web_md')

# Get the word vector for the word "King"
nlp("King").vector

In [ ]:
# Get the size of the vector
nlp("King").vector

In [ ]:
# Get the similarity between the two words "King" and "Queen"
nlp("King").similarity(nlp("Queen"))

In [ ]:
# Get the similarity between the two words "King" and "kangaroo"
nlp("King").similarity(nlp("kangaroo"))

# Introduction to Machine Learning 

Machine learning is a branch of artificial intelligence. Traditionally the human writes the rules in a computer system to perform a specific task. In machine learning, we use statistics to write the rules for us.

## Supervised Learning

**Supervised learning** is the process by which a system learns from a set of inputs that have known labels. To train the model, we will need part of the data for training, part for validation and part for testing. A rule of thumb to divide the input data into these categories is:

* 20% of all annotated data for testing
* For the remaining annotated data
    * 80% for training
    * 20% for validation

### Training and validation

The training data is used to hone a statistical model via predetermined algorithms. It does this by making guesses about what the proper labels are. It then checks its accuracy against the correct labels, i.e., the annotated labels, and makes adjustments accordingly.

Once it is finished viewing and guessing across all the training data, the first **epoch**, or **iteration** over the data, is finished. At this stage, the model then tests its accuracy against the validation data. The training data is then randomized and given back to the system for x number of epochs. Again, there is no standard for the number of epochs, but a good rule of thumb is to start at 10 and study the results.

### Testing
Once the model repeats this process for the set number of epochs, it is finished training. The model’s accuracy can then be tested against the testing dataset to see how well it performs.

# Create a custom model in SpaCy

In this section, we are going to go throught the process of creating a custom ML NER model in spaCy. 

First, let's download the two data files needed for this example. 

In [ ]:
# Download the data files
import urllib.request
urls = [
    'https://ithaka-labs.s3.amazonaws.com/static-files/images/tdm/tdmdocs/NER_HarryPotter_FilmSpells.csv',
    'https://ithaka-labs.s3.amazonaws.com/static-files/images/tdm/tdmdocs/NER_HarryPotter_Spells.csv',
]

for url in urls:
    urllib.request.urlretrieve(url, './data/' + url.rsplit('/', 1)[-1])   
print('Sample files ready.')

The first file stores the information about the spells in Harry Potter. 

In [ ]:
import pandas as pd
spells_df = pd.read_csv('./data/NER_HarryPotter_Spells.csv', sep=";")
spells_df

In the second file, we find the characters speaking and their speech. Notice that there is a column storing the spells found in the sentence if there is one. 

In [ ]:
film_spells = pd.read_csv('./data/NER_HarryPotter_FilmSpells.csv')
film_spells

Suppose we would like to create a model that can identify spells in a sentence.

In the following, we will first create a NLP model with an entity ruler that identifies spells. This section can be seen as a review of what we have learned about EntityRuler in Wednesday's lesson.

Then, we will train a NLP model using word2vec technique that identifies spells.

## Create an NLP model with an EntityRuler to identify the spells

Before we create a new EntityRuler, we will do some preprocessing of the data to get the patterns that we will add to the EntityRuler.

### Preprocessing the data

In [ ]:
# Fill the NaN cells with an empty string
spells_df['Incantation'] = spells_df['Incantation'].fillna("")

# Get all spells
spells = spells_df['Incantation'].unique().tolist() # Put all strs in the 'Incantation' column in a list
spells = [spell for spell in spells if spell != ''] # Get all non-empty strs from the list, i.e. all the spells

# Take a look at the spells
spells

### Creating the patterns to be added to the EntityRuler

Recall from Wednesday's lesson that the patterns we add to an EntityRuler look like the following.

`patterns = [{"label": "GPE", "pattern": "Aars"}]`

In [ ]:
patterns = []
for spell in spells:  
    spell_dict = {"label": "SPELL", "pattern": spell}
    patterns.append(spell_dict)
patterns

Now that we have the patterns ready, we can add them to an EntityRuler and add the ruler as a new pipe. 

In [ ]:
# Create an EntityRuler and add the patterns to the ruler
entruler_nlp = spacy.blank('en') # Create a blank English model
ruler = entruler_nlp.add_pipe("entity_ruler") 
ruler.add_patterns(patterns)

In [ ]:
test_text = """Ron Weasley: Wingardium Leviosa! Hermione Granger: You're saying it wrong. 
It's Wing-gar-dium Levi-o-sa, make the 'gar' nice and long. 
Ron Weasley: You do it, then, if you're so clever"""
doc = entruler_nlp(test_text)
for ent in doc.ents:
    print('EntRulerModel', ent.text, ent.label_)

## Train a NLP model using ML to identify the spells

We have basically hard written all spells in the EntityRuler we add to the entruler model. Therefore, we could just use this model to generate labeled data as our training data and validation data.

The format of the training data will look like the following. It is a list of tuples. In each tuple, the first element is the text string containing spells and the second element is a dictionary. The key of the dictionary is 'entities'. The value is a list of lists. In each list, we find the starting index, ending index and the label of the spell(s) found in the text string. 

`[
('Oculus Reparo.', {'entities': [[0, 13, 'SPELL']]}),
('Alohomora', {'entities': [[0, 9, 'SPELL']]})
]`

The text strings we use for the training are from the 'Sentence' column of the film_spells dataframe.

In [ ]:
# Take a look at the film_spells df
film_spells

In [ ]:
import nltk # for sentence tokenization
def generate_labeled_data(text): # the input will be a list of strings
    labeled_data = []
    for ele in text:
        sentences = nltk.sent_tokenize(ele)
        for sentence in sentences:
            doc = entruler_nlp(sentence) # create a doc object
            if doc.ents != (): # if there is at least one entity identified
                # for each entity that has been identified, generate a list containing
                # the starting index, the ending index and the label for the entity
                # these lists are put in a big list which is assigned to the variable ls_ents
                ls_ents = [[ent.start_char, ent.end_char, ent.label_] for ent in doc.ents] 
                entry = (sentence,{"entities": ls_ents})
                labeled_data.append(entry)
    return labeled_data

# Assign the result from the function to a new variable
training_validation_data = generate_labeled_data(film_spells['Sentence'].tolist())

# Take a look at the labeled data
training_validation_data

In [ ]:
training_validation_data

spaCy 3 requires that our data be stored in the proprietary `.spacy` format. To do that we need to use the `DocBin` class.

In [ ]:
from spacy.tokens import DocBin 

db = DocBin() 

for text, annot in training_validation_data[:19*2]: # Get the first 38 tuples as the training data
    doc = entruler_nlp(text) # create a doc object
    doc.ents = [doc.char_span(ent[0], ent[1], label=ent[2]) for ent in annot['entities']]
    db.add(doc)
db.to_disk(f"./train_spells.spacy")

In [ ]:
for text, annot in training_validation_data[19*2:]: # Get the rest tuples as the training data
    doc = entruler_nlp(text) 
    doc.ents = [doc.char_span(ent[0], ent[1], label=ent[2]) for ent in annot['entities']]
    db.add(doc)
db.to_disk(f"./valid_spells.spacy")

Now we can finally start training our model! 

In [ ]:
!python3 -m spacy init config --lang en --pipeline ner config.cfg --force

In [ ]:
!python3 -m spacy train config.cfg --output ./output/spells-model/ --paths.train ./train_spells.spacy --paths.dev ./valid_spells.spacy

Now let's finally run our model!

In [ ]:
model_best = spacy.load('./output/spells-model/model-best')
model_best.pipe_names

In [ ]:
test_text = """53. Imperio - Makes target obey every command But only for really, really funny pranks. 52. Piertotum Locomotor - Animates statues On one hand, this is awesome. On the other, someone would use this to scare me.

51. Aparecium - Make invisible ink appear

Your notes will be so much cooler.

50. Defodio - Carves through stone and steel

Sometimes you need to get the eff out of there.

49. Descendo - Moves objects downward

You'll never have to get a chair to reach for stuff again.

48. Specialis Revelio - Reveals hidden magical properties in an object

I want to know what I'm eating and if it's magical.

47. Meteolojinx Recanto - Ends effects of weather spells

Otherwise, someone could make it sleet in your bedroom forever.

46. Cave Inimicum/Protego Totalum - Strengthens an area's defenses

Helpful, but why are people trying to break into your campsite?

45. Impedimenta - Freezes someone advancing toward you

"Stop running at me! But also, why are you running at me?"

44. Obscuro - Blindfolds target

Finally, we don't have to rely on "No peeking."

43. Reducto - Explodes object

The "raddest" of all spells.

42. Anapneo - Clears someone's airway

This could save a life, but hopefully you won't need it.

41. Locomotor Mortis - Leg-lock curse

Good for footraces and Southwest Airlines flights.

40. Geminio - Creates temporary, worthless duplicate of any object

You could finally live your dream of lying on a bed of marshmallows, and you'd only need one to start.

39. Aguamenti - Shoot water from wand

No need to replace that fire extinguisher you never bought.

38. Avada Kedavra - The Killing Curse

One word: bugs.

37. Repelo Muggletum - Repels Muggles

Sounds elitist, but seriously, Muggles ruin everything. Take it from me, a Muggle.

36. Stupefy - Stuns target

Since this is every other word of the "Deathly Hallows" script, I think it's pretty useful."""

# Create a doc object out of the text string using the trained model
doc = model_best(test_text)

# Find out the entities
for ent in doc.ents:
    print(ent.text, ent.label_)

In [ ]:
# Create a doc object out of the text string using the EntityRuler model
doc = entruler_nlp(test_text)

# Find out the entities
for ent in doc.ents:
    print(ent.text, ent.label_)

It seems in this example our EntityRuler model performs better than our trained model. Why do we think that is?

Part of the reason we aren't getting better results is something that Ines Montani describes in this Stack Overflow answer https://stackoverflow.com/questions/50580262/how-to-use-spacy-to-create-a-new-entity-and-learn-only-from-keyword-list/50603247#50603247

"The advantage of training the named entity recognizer to detect SPECIES in your text is that the model won't only be able to recognise your examples, but also generalise and recognise other species in context. If you only want to find a fixed set of terms and not more, a simpler, rule-based approach might work better for you. You can find examples and details of this here.

If you do want the model to generalise and recognise your entity type in context, you also have to show it examples of the entities in context. That's currently the problem with your training examples: you're only showing the model single words, not sentences containing the words. To get good results, the data you're training the model with needs to be as close as possible to the data you later want to analyse.

While there are other approaches for training models without or with fewer labelled examples, the most straightforward strategy for collecting training data to train your spaCy model is to... label training data. However, there are some tricks you can use to make this less painful:

Start with a list of species and use the Matcher or PhraseMatcher to find them in your documents. For each match, you'll get a Span object, so you can extract the start and end position of the span in the text. This easily lets you create a bunch of examples automatically. You can find some more details on this here.

Use word vectors to find more similar terms to the entities you're looking for, so you get more examples you can search for in your text using the above approach. I'm not sure how spaCy's vector models will do for your species, since the terms are quite specific. So if you have a large corpus of raw text containing species, you might have to train your own vectors.

Use a labelling or data annotation tool. There are open-source solutions like Brat, or, once you're getting more serious about annotation and training, you might also want to check out our annotation tool Prodigy, which is a modern commercial solution that integrates seamlessly with spaCy (Disclaimer: I'm one of the spaCy maintainers)."

# References
McCormick, C. (2016, April 19). Word2Vec Tutorial - The Skip-Gram Model. Retrieved from http://www.mccormickml.com